In [1]:
import cv2
import torch
from pathlib import Path
import requests

class BlackjackDetector:
    """Klasa do detekcji kart i obliczania wyników w grze blackjack."""
    
    def __init__(self, model_url, camera_url, wait_time=5):
        self.model_url = model_url
        self.droidcam_url = camera_url
        self.model = self.download_model(model_url)
        self.wait_time = wait_time
        self.setup_roi()

    def download_model(self, model_url):
        model_path = Path("models/best.pt")
        if not model_path.is_file():
            model_path.parent.mkdir(parents=True, exist_ok=True)
            response = requests.get(model_url)
            response.raise_for_status()
            with open(model_path, 'wb') as f:
                f.write(response.content)
        return torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

    def setup_roi(self):
        """Konfiguracja regionów zainteresowania (ROI) dla każdego gracza i krupiera."""
        self.roi = {
            'player1': self.convert_to_roi_tuple([(200, 70), (368, 108), (78, 236), (266, 294)]),
            'player2': self.convert_to_roi_tuple([(368, 108), (544, 147), (266, 294), (471, 355)]),
            'player3': self.convert_to_roi_tuple([(544, 147), (771, 202), (471, 355), (755, 444)]),
            'dealer': self.convert_to_roi_tuple([(68, 251), (753, 466), (2, 354), (695, 594)])
        }

    @staticmethod
    def convert_to_roi_tuple(roi):
        """Konwersja współrzędnych ROI na krotkę."""
        x_min = min(point[0] for point in roi)
        y_min = min(point[1] for point in roi)
        x_max = max(point[0] for point in roi)
        y_max = max(point[1] for point in roi)
        return (x_min, y_min, x_max, y_max)

    @staticmethod
    def is_inside_roi(box, roi):
        """Sprawdzenie, czy dany obszar (box) znajduje się wewnątrz ROI."""
        x_min, y_min, x_max, y_max = box
        roi_x_min, roi_y_min, roi_x_max, roi_y_max = roi
        return (x_min >= roi_x_min and y_min >= roi_y_min and x_max <= roi_x_max and y_max <= roi_y_max)

    @staticmethod
    def is_corner_inside_main_card(corner_box, main_card_box):
        """Sprawdzenie, czy róg karty znajduje się wewnątrz głównej karty."""
        corner_x_min, corner_y_min, corner_x_max, corner_y_max = corner_box
        main_x_min, main_y_min, main_x_max, main_y_max = main_card_box
        return (corner_x_min >= main_x_min and corner_y_min >= main_y_min and corner_x_max <= main_x_max and corner_y_max <= main_y_max)

    @staticmethod
    def calculate_score(cards):
        """Oblicza wynik na podstawie listy kart."""
        values = {'j': 10, 'q': 10, 'k': 10, 'a': 11}
        score, aces = 0, 0
        for card in cards:
            if card in values:
                score += values[card]
            else:
                score += int(card)
            if card == 'a':
                aces += 1
    
        while score > 21 and aces:
            score -= 10
            aces -= 1
    
        return score


    @staticmethod
    def determine_result(players_scores, dealer_score):
        """Ustalenie wyniku gry na podstawie punktów graczy i krupiera."""
        results = {}
        for player, score in players_scores.items():
            if score == 21 and len(players_scores[player]) == 2:
                results[player] = "BLACKJACK"
            elif score > 21:
                results[player] = "za dużo"
            else:
                if dealer_score > 21 or (score <= 21 and score > dealer_score):
                    results[player] = "wygrana"
                elif score == dealer_score:
                    results[player] = "remis"
                else:
                    results[player] = "przegrana"
        return results

    def run_detection(self):
        """Uruchomienie procesu detekcji kart i obliczania wyników."""
        cap = cv2.VideoCapture(self.droidcam_url)
        
        continue_detection = True
    
        while continue_detection:
            ret, frame = cap.read()
            if not ret:
                print("Nie udało się uzyskać obrazu z kamery.")
                break
    
            frame = cv2.resize(frame, (800, 600))
            results = self.model(frame)
            results.render()
            player_cards = self.detect_cards(results)
    
            players_scores = self.calculate_players_scores(player_cards)
            self.display_results(players_scores)
    
            key = cv2.waitKey(0)
            break

    
        cap.release()
        cv2.destroyAllWindows()


    def detect_cards(self, results):
        """Detekcja kart dla każdego gracza i krupiera."""
        player_cards = {player: [] for player in self.roi}
        for *xyxy, conf, cls in results.xyxy[0]:
            box = [int(x) for x in xyxy]
            card_label = results.names[int(cls)]
            for player, roi in self.roi.items():
                if self.is_inside_roi(box, roi):
                    player_cards[player].append((box, card_label))
                    break
        return player_cards

    def calculate_players_scores(self, player_cards):
        """Obliczenie wyników dla każdego gracza."""
        players_scores = {}
        for player, cards in player_cards.items():
            identified_cards = self.identify_cards(cards)
            score = self.calculate_score(identified_cards)
            players_scores[player] = score
            self.display_player_cards(player, identified_cards, score)
        dealer_score = players_scores.pop('dealer')
        return self.determine_result(players_scores, dealer_score)

    def identify_cards(self, cards):
        """Identyfikacja kart na podstawie detekcji."""
        identified_cards = []
        for main_box, main_label in cards:
            if main_label.isdigit() or main_label in ['j', 'q', 'k', 'a']:
                valid_corners = [corner_label for corner_box, corner_label in cards if corner_label.startswith(main_label) and self.is_corner_inside_main_card(corner_box, main_box)]
                if valid_corners:
                    identified_cards.append(main_label)
        return identified_cards

    def display_player_cards(self, player, cards, score):
        """Wyświetlanie kart i wyniku danego gracza."""
        result_str = ', '.join(cards)
        result_str += " - wynik: " + ("BLACKJACK" if score == 21 and len(cards) == 2 else str(score))
        if score > 21:
            result_str += ", za dużo"
        print(f"{player.capitalize()} posiada karty: {result_str}")

    def display_results(self, players_scores):
        """Wyświetlanie końcowych wyników gry."""
        for player, result in players_scores.items():
            print(f"{player.capitalize()}: {result}")

In [2]:
# URL modelu (może być lokalny lub zdalny)
model_url = ''path/to/model'' #uzupełnić

# URL kamery
camera_url = "http://url/video" #uzupełnić

# Tworzenie instancji detektora
detector = BlackjackDetector(model_url, camera_url)

# Uruchomienie detekcji
detector.run_detection()

SyntaxError: invalid syntax (966806315.py, line 2)